# Development environment setup

## Libraries

In [90]:
import duckdb as db
import pandas as pd
import plotly.express as px
import sys

from importlib import reload
from pathlib import Path
from pprint import pprint


## Project path

In [91]:
# Resolve path to project root
project_path = Path("__file__").resolve().parents[1]

In [92]:
# Add project paht to sys.path
sys.path.append(str(project_path))

## Custom functions

In [93]:
from src import utils

## Parameters

In [94]:
prm = utils.load_parameters()

pprint(prm)

{'plotly': {'render': 'vscode+pdf+png+jpeg', 'theme': 'plotly_white'}}


# Data

## Product

In [95]:
product_dtypes = {
    "product_id": int,
    "producer_id": int,
    "registry_date": str,
    "recovery_active": int,
    "member_area_active": int,
    "deletion_date": str,
    "niche": str,
    "base_price": str,
    "type": str,
}

product = pd.read_csv(
    project_path.joinpath("data", "raw", "Product.csv"),
    dtype=product_dtypes,
)

# Some conversions have to be done after reading the data
# Convert registry_date from string to datetime
product['registry_date'] = product['registry_date'].apply(utils.parse_date)

# Convert deletion_date from string to datetime
product['deletion_date'] = product['deletion_date'].apply(utils.parse_date)

# Convert base_price from string to float by removing commas
product['base_price'] = product['base_price'].str.replace(',', '').astype(float)

product.head().T

,0,1,2,3,4
product_id,218834610,253458328,466262361,613311395,528943857
producer_id,1,4,6,2,2
registry_date,2020-03-30,2021-05-24,2020-11-13,2018-02-28,2018-09-15
recovery_active,1,1,1,0,1
member_area_active,1,1,1,0,0
deletion_date,NaT,NaT,NaT,NaT,NaT
niche,Marketing,Educação Infantil,Empreendedorismo,Financas,Finanças
base_price,780.0,560.0,800.0,680.0,500.0
type,Curso,Assinatura,Curso,Curso,Curso


The niche column has 2 typos that we will fix now:

- Tecnologia-e-Inovação: Tecnologia e Inovação
- Financas: Finanças

In [96]:
# Fixing typos in the data
typos_fix = {
    "Tecnologia-e-Inovação": "Tecnologia e Inovação",
    "Financas": "Finanças"
}

product["niche"] = product["niche"].replace(typos_fix)

## Producer

In [97]:
producer_dtypes = {
    "producer_id": int,
    "registry_date":str,
    "country": str
}

producer = pd.read_csv(
    project_path.joinpath("data", "raw", "Producer.csv"),
    dtype=producer_dtypes
)

# Some conversions have to be done after reading the data
# Convert registry_date from string to datetime
producer['registry_date'] = producer['registry_date'].apply(utils.parse_date)

producer.head().T

,0,1,2,3,4
producer_id,1,2,3,4,5
registry_date,2018-02-01,2018-01-04,2020-01-02,2021-01-02,2019-02-11
country,BRASIL,COLOMBIA,BRASIL,COLOMBIA,BRASIL


## Sales

In [98]:
sales_dtypes = {
    "purchase_id": int,
    "purchase_date": str,
    "product_id": int,
    "product_price": str,
    "service_tax": str,
    "comission_value": str,
    "has_coupon": int,
    "discount": str,
    "refund": int,
    "cancelled": int,
    "chargeback": int,
}

sales = pd.read_csv(
    project_path.joinpath("data", "raw", "Sales.csv"),
    dtype=sales_dtypes
)

# Some conversions have to be done after reading the data
# Convert purchase_date from string to datetime
sales['purchase_date'] = sales['purchase_date'].apply(utils.parse_date)

# Convert product_price from string to float by removing commas
sales['product_price'] = sales['product_price'].str.replace(',', '').astype(float)

# Convert service_tax from string to float by removing commas
sales['service_tax'] = sales['service_tax'].str.replace(',', '').astype(float)

# Convert comission_value from string to float by removing commas
sales['comission_value'] = sales['comission_value'].str.replace(',', '').astype(float)

# Convert discount from string to float by removing % and dividing by 100
sales['discount'] = sales['discount'].str.replace('%', '').astype(float) / 100

sales.head().T

,0,1,2,3,4
purchase_id,546791636,863821559,342304284,421394541,596413254
purchase_date,2020-03-18,2021-09-15,2021-08-28,2021-08-15,2019-12-22
product_id,903828613,264261718,382406448,218834610,603531440
product_price,531.0,560.0,480.0,780.0,406.0
service_tax,53.1,56.0,62.4,101.4,40.6
comission_value,477.9,504.0,417.6,0.0,365.4
has_coupon,1,0,0,0,1
discount,0.1,0.0,0.0,0.0,0.3
refund,0,0,0,0,0
cancelled,0,0,0,0,0


# Technical case

## 1) The top 10 products that sold the most in each niche with deactivated membership area and activated recovery

In [99]:
query = """
WITH sales_count AS (
    SELECT
        p.niche,
        p.product_id,
        COUNT(s.purchase_id) AS total_sales
    FROM
        product AS p
    INNER JOIN
        sales AS s ON p.product_id = s.product_id
    WHERE
        p.member_area_active = 0
        AND
        p.recovery_active = 1
    GROUP BY
        ALL
),

sales_ranking AS (
    SELECT
        sc.*,
        RANK() OVER (PARTITION BY sc.niche ORDER BY sc.total_sales DESC) AS niche_rank
    FROM
        sales_count AS sc
)

SELECT
    sr.niche,
    sr.product_id,
    sr.total_sales,
    sr.niche_rank
FROM
    sales_ranking AS sr
WHERE
    sr.niche_rank <= 10
ORDER BY
    sr.niche,
    sr.niche_rank
"""

top_10_products_niche = db.query(query).to_df()

top_10_products_niche.head(30)

,niche,product_id,total_sales,niche_rank
0,Artes e Design,273593751,54,1
1,Empreendedorismo,983844542,77,1
2,Empreendedorismo,844949729,55,2
3,Empreendedorismo,382406448,37,3
4,Empreendedorismo,954425664,1,4
5,Empreendedorismo,555755392,1,4
6,Finanças,603531440,53,1
7,Finanças,264261718,18,2
8,Finanças,348532296,3,3
9,Finanças,204306513,2,4


## 2) The top 10 producers who joined XPCourses from 2020 onwards and achieved the highest sales using recovery.

Here we'll use the **comission value** of each sale to rank the producers.

All sales with chargeback, refund and cancelled flags will have $0,00 comission value.

In [100]:
query = """
WITH producers_sales AS (
    SELECT
        pr.producer_id,
        COUNT(DISTINCT s.purchase_id) AS total_sales,
        SUM(s.comission_value) AS total_comission
    FROM
        producer AS pr
    INNER JOIN
        product AS p ON pr.producer_id = p.producer_id
    INNER JOIN
        sales AS s ON p.product_id = s.product_id
    WHERE
        YEAR(pr.registry_date) >= 2020
    GROUP BY
        pr.producer_id
),

producers_ranking AS (
    SELECT
        ps.*,
        RANK() OVER (ORDER BY ps.total_comission DESC) AS producer_rank
    FROM
        producers_sales AS ps
)

SELECT
    pr_r.producer_id,
    pr_r.total_sales,
    pr_r.total_comission,
    pr_r.producer_rank
FROM
    producers_ranking AS pr_r
WHERE
    pr_r.producer_rank <= 10
ORDER BY
    pr_r.producer_rank
"""

top_10_producers = db.query(query).to_df()

top_10_producers

,producer_id,total_sales,total_comission,producer_rank
0,3,181,77343.37,1
1,7,80,37754.65,2
2,10,59,19052.45,3
3,4,14,7136.80,4


## 3) How much more a producer with the recovery feature activated is likely to sell in each niche? Consider only producers who registered from 2020 onwards.

Although we cannot say that the recovery feature is the **cause** of the increase in sales, we will calculate the average of all sales values ​​of products in each niche with and without this feature and then compare this value.

In [101]:
query = """
WITH avg_per_product_niche AS (
    SELECT
        p.niche,
        (SUM(s.product_price) FILTER (p.recovery_active = 0)) / (COUNT(DISTINCT p.product_id) FILTER (p.recovery_active = 0)) AS avg_price_without_recovery,
        (SUM(s.product_price) FILTER (p.recovery_active = 1)) / (COUNT(DISTINCT p.product_id) FILTER (p.recovery_active = 1)) AS avg_price_with_recovery,
    FROM
        producer AS pr
    INNER JOIN
        product AS p ON pr.producer_id = p.producer_id
    INNER JOIN
        sales AS s ON p.product_id = s.product_id
    WHERE
        YEAR(pr.registry_date) >= 2020
    GROUP BY
        p.niche
)

SELECT
    a.niche,
    a.avg_price_without_recovery,
    a.avg_price_with_recovery,
    a.avg_price_with_recovery - a.avg_price_without_recovery AS recovery_gain,
    a.avg_price_with_recovery / a.avg_price_without_recovery AS recovery_gain_pct
FROM
    avg_per_product_niche AS a
ORDER BY
    a.niche
"""

recovery_agin = db.query(query).to_df()

recovery_agin


,niche,avg_price_without_recovery,avg_price_with_recovery,recovery_gain,recovery_gain_pct
0,Artes e Design,NaN,11017.0,NaN,NaN
1,Educação Infantil,NaN,4403.0,NaN,NaN
2,Empreendedorismo,NaN,34183.5,NaN,NaN
3,Finanças,NaN,2040.0,NaN,NaN
4,Saúde e Alimentação,NaN,6604.0,NaN,NaN
5,Tecnologia e Inovação,NaN,26705.0,NaN,NaN


In [102]:
query = """
SELECT
    pr.producer_id,
    pr.registry_date,
    p.niche,
    p.recovery_active
FROM
    producer AS pr
INNER JOIN
    product AS p ON pr.producer_id = p.producer_id
WHERE
    YEAR(pr.registry_date) >= 2020
ORDER BY
    p.niche,
    p.recovery_active
"""

db.query(query).to_df()

,producer_id,registry_date,niche,recovery_active
0,10,2020-01-12,Artes e Design,1
1,10,2020-01-12,Artes e Design,1
2,4,2021-01-02,Educação Infantil,1
3,4,2021-01-02,Educação Infantil,1
4,4,2021-01-02,Educação Infantil,1
5,4,2021-01-02,Educação Infantil,1
6,3,2020-01-02,Empreendedorismo,1
7,3,2020-01-02,Empreendedorismo,1
8,10,2020-01-12,Finanças,1
9,4,2021-01-02,Idiomas,1


All producers registered in 2020 onwards **only have products with the recovery feature activated**, so it is not possible to calculate the gain from this feature.

## 4) The product niche(s) with the highest number of cancellations and refunds.

In [103]:
query = """
SELECT
    p.niche,
    100 * (COUNT(DISTINCT s.purchase_id) FILTER (s.cancelled = 1) / COUNT(DISTINCT s.purchase_id)) AS pct_cancelled,
    100 * (COUNT(DISTINCT s.purchase_id) FILTER (s.refund = 1) / COUNT(DISTINCT s.purchase_id)) AS pct_refunded,
FROM
    product AS p
INNER JOIN
    sales AS s ON p.product_id = s.product_id
GROUP BY
    p.niche
ORDER BY
    p.niche
"""

pcts_cancelled_and_refunded = db.query(query).to_df()

pcts_cancelled_and_refunded

,niche,pct_cancelled,pct_refunded
0,Artes e Design,5.357143,5.357143
1,Educação Infantil,0.000000,0.000000
2,Empreendedorismo,2.673797,5.347594
3,Finanças,2.020202,5.050505
4,Marketing,4.280156,1.945525
5,Saúde e Alimentação,3.658537,6.097561
6,Tecnologia e Inovação,8.139535,3.100775
7,Viagens,2.173913,8.695652


In [104]:
top_cancelled = pcts_cancelled_and_refunded.sort_values("pct_cancelled", ascending=False).head(1)
top_refunded = pcts_cancelled_and_refunded.sort_values("pct_refunded", ascending=False).head(1)

print(f"The '{top_cancelled.niche.values[0]}' niche has the highest percentage of cancelled sales: {top_cancelled.pct_cancelled.values[0]:.2f}%")
print(f"The '{top_refunded.niche.values[0]}' niche has the highest percentage of refunded sales: {top_refunded.pct_refunded.values[0]:.2f}%")

The 'Tecnologia e Inovação' niche has the highest percentage of cancelled sales: 8.14%
The 'Viagens' niche has the highest percentage of refunded sales: 8.70%


## 5) Calculate the total money lost by producer due to cancellations and refunds. Is there any difference for producers considering products with the recovery tool activated?

In [105]:
query = """
SELECT
    p.producer_id,
    SUM(s.product_price::FLOAT) AS total_sales,
    SUM(s.product_price::FLOAT) FILTER(p.recovery_active = 1) AS total_sales_with_recovery,
    SUM(s.product_price::FLOAT) FILTER(p.recovery_active = 0) AS total_sales_without_recovery,
    SUM(s.product_price::FLOAT) FILTER(s.cancelled = 1 OR s.refund = 1) AS total_sales_lost,
    SUM(s.product_price::FLOAT) FILTER((s.cancelled = 1 OR s.refund = 1) AND p.recovery_active = 1) AS total_sales_lost_with_recovery,
    SUM(s.product_price::FLOAT) FILTER((s.cancelled = 1 OR s.refund = 1) AND p.recovery_active = 0) AS total_sales_lost_without_recovery,
FROM
    product AS p
INNER JOIN
    sales AS s ON p.product_id = s.product_id
GROUP BY
    p.producer_id
ORDER BY
    p.producer_id
"""

money_lost = db.query(query).to_df()

money_lost

,producer_id,total_sales,total_sales_with_recovery,total_sales_without_recovery,total_sales_lost,total_sales_lost_with_recovery,total_sales_lost_without_recovery
0,1,186557.0,186557.0,NaN,10316.0,10316.0,NaN
1,2,48770.0,48090.0,680.0,5376.0,5376.0,NaN
2,3,93015.0,93015.0,NaN,5265.0,5265.0,NaN
3,4,8806.0,8806.0,NaN,NaN,NaN,NaN
4,5,48151.0,48151.0,NaN,4566.0,4566.0,NaN
5,6,45005.0,45005.0,NaN,1470.0,1470.0,NaN
6,7,48574.0,48574.0,NaN,5782.0,5782.0,NaN
7,8,90924.0,33580.0,57344.0,9924.0,2812.0,7112.0
8,9,13531.0,13531.0,NaN,1421.0,1421.0,NaN
9,10,24074.0,24074.0,NaN,2419.0,2419.0,NaN


In [109]:
top_producer_lost = money_lost.sort_values("total_sales_lost", ascending=False).head(1)

print(f"The 'producer_id={top_producer_lost.producer_id.values[0]}' has the highest amount lost due cancellation and refunds: ${top_producer_lost.total_sales_lost.values[0]:,.2f}")

The 'producer_id=1' has the highest amount lost due cancellation and refunds: $10,316.00


Analyzing the previous table, the only producer that has recovery disabled on some of its products and has suffered losses due to cancellations and refunds is 'producer_id=8', so let's analyze it.


In [113]:
producer_8 = money_lost.query("producer_id == 8")

print(f"The 'producer_id={producer_8.producer_id.values[0]}' has lost ${producer_8.total_sales_lost.values[0]:,.2f} due cancellation and refunds, with {100 * producer_8.total_sales_lost_without_recovery.values[0] / producer_8.total_sales_lost.values[0]:.2f}% in products without recovery feature")

The 'producer_id=8' has lost $9,924.00 due cancellation and refunds, with 71.66% in products without recovery feature


## 6) If you need to create a ranking of the top creators of 2023, which variables you consider crucial for ranking them? You can also create variables from the data. You must explain your reasoning and your choice of variables and show how this reflect in your SQL code.